In [51]:
import itertools
import json
import pickle
import re

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

from tqdm.notebook import tqdm
tqdm.pandas()

In [52]:
with open("../../data/pseudowords/CoMaPP_all.json") as json_file:
    data = json.load(json_file)
    
data = [{"example": d["target1"], "cue": " ".join(d["target1"].split()[:d["query_idx"]]), "pseudoword": d["label"]} for d in data if d["target1"].split()[d["query_idx"]] in d["label"]]
df = pd.DataFrame.from_dict(data).drop_duplicates(ignore_index=True)
df

,example,cue,pseudoword
0,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
1,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
2,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,am488
3,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,wenigsten488
4,Unter den Einwohnern der großen EU - Länder si...,Unter den Einwohnern der großen EU - Länder si...,am488
...,...,...,...
8503,"Protestzüge wälzen sich durch München , Eisner...",Protestzüge wälzen,sich1573
8504,Und in Sands bewege sich ja einiges .,Und in Sands bewege,sich1573
8505,Ulrike beugt sich über die Partitur .,Ulrike beugt,sich1573
8506,"Am Anfang gab es viel Widerstand , man holpert...","Am Anfang gab es viel Widerstand , man holperte",sich1573


In [53]:
df['index'] = df['pseudoword'].str.extract('(\d+)').astype(int)
df.set_index('index', inplace=True)

df

,example,cue,pseudoword
index,,,
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,am488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,wenigsten488
488,Unter den Einwohnern der großen EU - Länder si...,Unter den Einwohnern der großen EU - Länder si...,am488
...,...,...,...
1573,"Protestzüge wälzen sich durch München , Eisner...",Protestzüge wälzen,sich1573
1573,Und in Sands bewege sich ja einiges .,Und in Sands bewege,sich1573
1573,Ulrike beugt sich über die Partitur .,Ulrike beugt,sich1573


In [54]:
contextleft = pd.read_pickle("../../data/pseudowords/contextleft_text.pickle")

def update_cue(row):
    output = row[['example', 'cue']]
    if row['cue'] == '':  # if the string in cue is empty
        # match the index of row with contextleft['construction_id'] and match contextleft['text'] with row['example'] and create matching_entry
        matching_entry = contextleft.loc[(contextleft['construction_id'] == row.name) & (contextleft['text'] == row['example']), 'contextleft'].tolist()
        if len(matching_entry) > 0:
            output = [matching_entry[0] + " " + row['example'], matching_entry[0]]
    return output

# Add the left context if there is no cue up until the pseudoword.
df[["example", "cue"]] = df.apply(update_cue, axis=1)
df

,example,cue,pseudoword
index,,,
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,am488
488,Doch der theatralische Selbstmord von General ...,Doch der theatralische Selbstmord von General ...,wenigsten488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,am488
488,Morgens beim Kaffee wird gemeinsam entschieden...,Morgens beim Kaffee wird gemeinsam entschieden...,wenigsten488
488,Unter den Einwohnern der großen EU - Länder si...,Unter den Einwohnern der großen EU - Länder si...,am488
...,...,...,...
1573,"Protestzüge wälzen sich durch München , Eisner...",Protestzüge wälzen,sich1573
1573,Und in Sands bewege sich ja einiges .,Und in Sands bewege,sich1573
1573,Ulrike beugt sich über die Partitur .,Ulrike beugt,sich1573


In [55]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'construction'}, inplace=True)

result_df = df.groupby(['construction', 'pseudoword']).agg({'example': list, 'cue': list})

result_df

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  
construction pseudoword                                                     
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...  
             erst5       [Trainer Lucien Favre hatte schon seine beiden...  
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...  
             nicht5      [Es hat Afghanistan, Dass es in der Familie nu...  
             recht5      [Trainer Lucien Favre hatte schon seine beiden...  
...                                                                    ...  
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...  
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...  
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...  
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...  
1987         sehr1987    [Solche Menschen können sich, " " Das hat die ...  

[506 rows x 2 columns]

In [56]:
with open("../../out/definitions.pickle", "rb") as definitions_file:
    definitions = pd.DataFrame.from_dict(pickle.load(definitions_file), orient="index", columns=["definition"])
    
definitions

,definition
677,"Bei der Konstruktion ""Additiv_Koordinativkompo..."
563,"Bei der ""Gegenüberstellung:V2_V2-Konstruktion""..."
696,"Die ""Konjunktiv:V.conj-I-Konstruktion"" gehört ..."
488,"Die ""Superlativ:am_meisten/wenigstenADJ-Konstr..."
674,"Die ""Kausaler_Konnektor:weil-Konstruktion"" die..."
...,...
875,"Bei der Konstruktion (Kxn) ""Kategorisierung_Tr..."
1323,Die meist als Prädikativum verwendete Konstruk...
85,"Die ""Reduplikation_Quantifizierung:N1_über_N1-..."
1792,"Bei der Konstruktion (Kxn) ""Prädikation_Negati..."


In [57]:
examples = pd.merge(result_df, definitions, how="inner", left_on="construction", right_index=True)
examples

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan, Dass es in der Familie nu...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...   
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...   
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich, " " Das hat die ...   

                                                                definition  
construction pseudoword                                                     
5            Und5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             erst5       Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             gar5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             nicht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             recht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
...                                                                    ...  
1884         Gold1884    Die "Intensivierung_Komparativ:Xist_SilberYist...  
             Silber1884  Die "Intensivierung_Komparativ:Xist_SilberYist...  
             ist1884     Die "Intensivierung_Komparativ:Xist_SilberYist...  
1986         kaum1986    Die Konstruktion "Relativierung:kaumADJ" gehör...  
1987         sehr1987    Die Konstruktion "Intensivierung:sehrV" gehört...  

[506 rows x 3 columns]

##### Generieren neuer Sätze:

In [58]:
pseudowords = [
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_0_69.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_69_93.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_93_165.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_165_176.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_176_281.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_281_364.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_364_437.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_437_562.npy")
]
pseudowords = np.concatenate(pseudowords)
pseudowords

array([[ 0.01952364,  0.00048042,  0.05060846, ...,  0.04258203,
         0.05651806,  0.02429085],
       [-0.00994319, -0.00676309, -0.00601289, ..., -0.03203335,
        -0.03284921,  0.00028247],
       [-0.03243406, -0.00804963, -0.02258722, ...,  0.04839412,
         0.0025557 ,  0.01301195],
       ...,
       [ 0.00162825,  0.01625418,  0.0080498 , ..., -0.01727665,
         0.01961022, -0.00274661],
       [ 0.00529211,  0.00459571,  0.01711997, ...,  0.01059076,
        -0.01628351, -0.0441331 ],
       [-0.03313114,  0.00591089, -0.00273072, ..., -0.03151482,
         0.01069049,  0.00970152]], dtype=float32)

In [59]:
csv_data = []
# TODO mBART-Order laden
for i in range(0, 8):
    csv_data.append(pd.read_csv(f"../../data/pseudowords/mbart/order_{i}.csv", sep=";", index_col=0, header=None, quotechar="|", names=["order", "label"]))
csv_data = pd.concat(csv_data)
csv_data

,label
order,
0,"""""Was13"
1,"""647"
3,(1597
4,(1600
5,(1602
...,...
556,»Raus1316
557,Ähnlich123
559,ähnlich123


In [60]:
mbart_tokens = [d[0] for d in csv_data.values]

mbart_tokens, len(mbart_tokens)

(['""Was13',
  '"647',
  '(1597',
  '(1600',
  '(1602',
  '(1637',
  '(1641',
  '(1643',
  '(379',
  '(579',
  '(581',
  '(584',
  '(590',
  '(592',
  '(600',
  '(886',
  '(892',
  '(900',
  '(905',
  '(907',
  '(909',
  '(917',
  '(919',
  '(921',
  ')1597',
  ')1600',
  ')1637',
  ')1641',
  ')1643',
  ')1792',
  ')379',
  ')579',
  ')581',
  ')584',
  ')590',
  ')592',
  ')600',
  ')886',
  ')892',
  ')900',
  ')907',
  ')909',
  ')917',
  ')919',
  ')921',
  ')«579',
  ',1459',
  ',973',
  '-128',
  '-651',
  '-654',
  '-875',
  '-973',
  ':595',
  ':875',
  ':973',
  'Abstand683',
  'Allein20',
  'Aller1630',
  'Als1315',
  'Als133',
  'Als1770',
  'Am488',
  'Am492',
  'Am500',
  'Amerika605',
  'genauso122',
  'genauso98',
  'geschweige10',
  'gewesen1459',
  'gibt605',
  'gleich100',
  'gleich675',
  'gleich676',
  'gleich98',
  'gleiche104',
  'gleichen1777',
  'gleichkam676',
  'gleichkommen676',
  'gleichkommt676',
  'gleichkäme676',
  'gleicht132',
  'gold1554',
  'habe605'

Load the vanilla mbart model:

In [61]:
model = MBartForConditionalGeneration.from_pretrained(
    "facebook/mbart-large-50", return_dict=True
) 
tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50", src_lang="de_DE", tgt_lang="de_DE"
)
model.model.encoder.embed_tokens

Embedding(250054, 1024, padding_idx=1)

Add to existing embeddings:

In [62]:
combined_embeddings = torch.cat((model.model.shared.weight, torch.tensor(pseudowords)), dim=0)
model.model.encoder.embed_tokens = torch.nn.Embedding.from_pretrained(combined_embeddings)
model.model.encoder.embed_tokens

Embedding(250560, 1024)

Add to existing tokens:

In [63]:
tokenizer.add_tokens(mbart_tokens)
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 250560. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(250560, 1024)

In [64]:
# This part simulates the behavior during training, preventing the Decoder from returning the pseudoword itself.
def freeze(model, new_embeds):
    # Freeze all the parameters except the word embeddings
    for name, param in model.named_parameters():
        if 'model.shared' in name:
            param.requires_grad = True
        else:
            param.requires_grad = False

    # The "decoder" in BERT maps from hidden to output. This is analogous to "lm_head" in mBART.
    original_weight = model.lm_head.weight
    original_bias = model.final_logits_bias
    # The vocabulary of the decoder doesn't need the new pseudoword tokens and would be too big:
    original_decoder_embed_tokens_weight = model.model.decoder.embed_tokens.weight

    # The argument len(tokenizer)-new_embeds should prevent the model from outputting the new tokens:
    lm_head = nn.Linear(in_features=1024, out_features=len(tokenizer) - new_embeds, bias=False)
    lm_head.weight.requires_grad = False
    model.register_buffer("final_logits_bias", torch.zeros((1, model.model.shared.num_embeddings - new_embeds)))
    lm_head.weight.data.copy_(original_weight.data[:-new_embeds])
    model.final_logits_bias.copy_(original_bias[:, :-new_embeds])
    model.lm_head = lm_head
    decoder_embed_tokens = nn.Embedding(len(tokenizer) - new_embeds, model.config.d_model, model.config.pad_token_id)
    # For decoder, see above:
    decoder_embed_tokens.weight.data.copy_(original_decoder_embed_tokens_weight.data[:-new_embeds])
    decoder_embed_tokens.requires_grad_(False)
    model.model.decoder.embed_tokens = decoder_embed_tokens
    model.config.vocab_size -= new_embeds

    return model

In [65]:
model = freeze(model, len(mbart_tokens))
model.to("cuda:0")

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250560, 1024)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250560, 1024)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05

Complete the cues:

In [66]:
def complete_cues(row):
    try:
        output_texts = []
        scores = []
        for cue, example in zip(row["cue"], row["example"]):
            assert row["pseudoword"].iloc[0] in mbart_tokens  # skip pseudoword embeddings that haven't been learned
            
            input_text = "de_DE <s>" + cue + " " + row["pseudoword"].iloc[0] + " <mask>" + "</s>"
            target_length = int(1.5 * len(example))  # allow double the length of the original sentence
            
            outputs = tokenizer(input_text, return_tensors="pt")
            outputs = model.generate(outputs["input_ids"].to("cuda:0"), max_length=target_length, num_return_sequences=1, num_beams=10, output_scores=True, return_dict_in_generate=True)
            output_text = tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)
            score = torch.exp(outputs.sequences_scores)
            output_texts += output_text
            
            scores.append(score)
        print(".", end="")
        return pd.Series({"construction": row["construction"], "pseudoword": row["pseudoword"].iloc[0], "orig_example": row["example"], "generated": str(output_texts), "scores": str([float(score) for score in scores])})
    except Exception as e:
        print(":", end="")
        return pd.Series({"construction": row["construction"], "pseudoword": row["pseudoword"].iloc[0], "orig_example": row["example"], "generated": str(e), "scores": "[-1.0]"})

examples_reset = examples.reset_index()
pseudoword_output_scores = examples_reset[["construction", "pseudoword", "example", "cue", "pseudoword"]][:20].progress_apply(complete_cues, axis=1)
pseudoword_output_scores

  0%|          | 0/20 [00:00<?, ?it/s]

...:................

,construction,pseudoword,orig_example,generated,scores
0,5,Und5,"[""""Nicht herauskaufen"""", """"nicht erpressen las...","['""""Nicht herauskaufen"""", """"nicht erpressen la...","[0.7198575735092163, 0.8319424390792847, 0.831..."
1,5,erst5,[Trainer Lucien Favre hatte schon seine beiden...,[' Trainer Lucien Favre hatte schon seine beid...,"[0.7870684266090393, 0.7485634088516235, 0.853..."
2,5,gar5,[Es hat Afghanistan nicht stabilisiert und sch...,[' Es hat Afghanistan nicht stabilisiert und s...,"[0.7306739687919617, 0.6657924652099609, 0.746..."
3,5,nicht5,[Es hat Afghanistan nicht stabilisiert und sch...,,[-1.0]
4,5,recht5,[Trainer Lucien Favre hatte schon seine beiden...,[' Trainer Lucien Favre hatte schon seine beid...,"[0.8094529509544373, 0.7195969820022583, 0.864..."
5,5,schon5,[Es hat Afghanistan nicht stabilisiert und sch...,[' Es hat Afghanistan nicht stabilisiert und ...,"[0.6981704831123352, 0.7472202181816101, 0.781..."
6,5,und5,[Es hat Afghanistan nicht stabilisiert und sch...,"[' Es hat Afghanistan nicht stabilisiert ',...","[0.741619884967804, 0.8281431198120117, 0.6814..."
7,10,Geschweige10,"[Die Chance dazu haben sie, und sie haben sie ...","['........', ' Sie kennen aber nicht ihren Kon...","[0.5573006868362427, 0.6917740702629089, 0.888..."
8,10,denn10,[Und dann ist da noch das generelle Problem mi...,[' Und dann ist da noch das generelle Problem ...,"[0.8530645966529846, 0.729907214641571, 0.7148..."
9,10,geschweige10,[Und dann ist da noch das generelle Problem mi...,['... Und dann ist da noch das generelle Probl...,"[0.7693580389022827, 0.70081627368927, 0.66258..."


In [17]:
examples = pseudoword_output_scores[["pseudoword", "generated", "scores"]]

examples

,pseudoword,generated,scores
0,Und5,"['""""Nicht herauskaufen"""", """"nicht erpressen la...","[0.9136524200439453, 0.44515761733055115, 0.51..."
1,erst5,['Trainer Lucien Favre hatte schon seine beide...,"[0.7875338792800903, 0.7785300612449646, 0.742..."
2,gar5,['Es hat Afghanistan nicht stabilisiert gar5 -...,"[0.5824133157730103, 0.6445226073265076, 0.719..."
3,nicht5,,[-1.0]
4,recht5,['Trainer Lucien Favre hatte schon seine beide...,"[0.7853999733924866, 0.8219645023345947, 0.791..."
...,...,...,...
557,Gold1884,"['Schweigen ist Silber, reden ist ', 'Schweige...","[0.7522048950195312, 0.8172914981842041, 0.784..."
558,Silber1884,"['Schweigen ist ', 'Schweigen ist ', 'Ange...","[0.7068747878074646, 0.7068747878074646, 0.643..."
559,ist1884,"['Schweigen ist1884 (Englisch)', 'Schweigen is...","[0.4845588505268097, 0.4845588505268097, 0.498..."
560,kaum1986,"['Die Vorhut vor 20.000 Jahren ', 'Vorhut vo...","[0.6681877374649048, 0.5497788190841675, 0.670..."


In [18]:
examples.to_csv(f"../../out/comapp/mbart/data_mbart.tsv", sep="\t", decimal=",")
examples.to_excel(f"../../out/comapp/mbart/data_mbart.xlsx")

In [19]:
pseudoword_output_scores.to_csv("../../out/comapp/mbart/data_mbart_complete.tsv", sep="\t", decimal=",")